In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
from PyQt4 import QtCore, QtGui
n_calc = [0]
def calculate():
    sin(rand(1000))
    n_calc[0]+=1
    timer.start()

timer = QtCore.QTimer()
timer.setInterval(0)
timer.setSingleShot(True)
timer.timeout.connect(calculate)

In [3]:
from PyQt4 import QtCore, QtGui
from psutil import cpu_percent
APP = QtGui.QApplication.instance()

close('all')

n_calc[0] = 0

from time import time
def sleep_loop(delay):
   loop = QtCore.QEventLoop()
   timer = QtCore.QTimer()
   timer.setInterval(delay*1000)
   timer.setSingleShot(True)
   timer.timeout.connect(loop.quit)
   timer.start()
   loop.exec() # la loop prend en charge elle-même l'évenement du timer qui va la faire mourir après delay.

def sleep_pe(delay):
   time0 = time()
   while(time()<time0+delay):
       APP.processEvents()

def my_sleep(delay):
    tic = default_timer()
    if delay>1e-3:
        sleep_loop(delay - 1e-3)
    while(default_timer()<tic+delay):
        pass
   
    
from time import sleep
from timeit import default_timer
MAX_TIME = 0.005

for timer_on_off in (False, True):
    for index, (func, label) in enumerate([(sleep_loop, "qeventloop"),
                        (sleep_pe, "processEvents"),
                        (sleep, "time.sleep"),
                        (my_sleep, "my_sleep"), 
                       ]):
        if timer_on_off:
            timer.start()
        fig = figure()
        times_measured = []
        times_asked = []

        n_calc_before = n_calc[0]
        tic_for_func = default_timer()
        cpu_percent() # call starts the cpu_percent averaging
        for time_asked in linspace(0., MAX_TIME, 200):
            #if int(time_asked*10000)%10==0:
            #    print(time_asked)
            for i in range(10):
                tic = default_timer()
                func(time_asked)
                toc = default_timer()
                times_measured.append(toc - tic)
                times_asked.append(time_asked)
        cpu_percents = cpu_percent()
        toc_for_func = default_timer()
        calc_done = (n_calc[0] - n_calc_before)/(toc_for_func - tic_for_func)
        plot(times_asked, times_measured, '.')
        plot([0, MAX_TIME], [0, MAX_TIME])
        xlabel("time asked")
        ylabel("time measured")
        xlim(0, MAX_TIME)
        ylim(0, MAX_TIME)
        title(label + ' cpu_percent=' + str(cpu_percents) + ' calc/s=' + str(int(calc_done)))
        if timer_on_off:
            dirname = './images/with_timer/'
        else:
            dirname = './images/without_timer/'
        fig.savefig(dirname + label + '.png')
        timer.stop()
        close('all')
        for i in range(1000000):
            APP.processEvents()
timer.stop()